In [291]:
import pandas as pd
import numpy as np
from numpy import dot
from numpy.linalg import norm
# from gensim.models.doc2vec import Doc2Vec, TaggedDocument
# !pip install gensim

In [292]:
# df = pd.read_excel('/content/drive/My Drive/빅데이터_콘테스트/data_information/bigcontest_record_train.xlsx', header=1)

df = pd.read_excel('C:\\Users\\user\\Documents\\2020빅콘테스트 문제데이터(데이터분석분야-챔피언리그)_update_200818\\01_제공데이터\\2020 빅콘테스트 데이터분석분야-챔피언리그_2019년 실적데이터_v1_200818.xlsx',header=1)

features = ['broadcast', 'exp_min', 'm_code', 'p_code', 'p_name', 'p_group', 'unit_price', 'total_price']
df.columns=features
# df['sales_amount'] = round(df['total_price']/df['unit_price'],1)

df['broadcast'] = df['broadcast'].astype('datetime64[ns]')
df['weekday'] = df.broadcast.dt.weekday + 1 # 월요일:1, 일요일:7
df['month'] = df.broadcast.dt.month
df['day'] = df.broadcast.dt.day
df['hour'] = df.broadcast.dt.hour
df['quarter'] = df.broadcast.dt.quarter
df = df[df.p_group != '무형']
df = df.reset_index(drop=True)

In [293]:
p_name_list = df.p_name.unique()
p_name_list

array(['테이트 남성 셀린니트3종', '테이트 여성 셀린니트3종', '오모떼 레이스 파운데이션 브라', ...,
       '램프쿡 자동회전냄비', '벨라홈 스마트 멀티포트 1+1 세트', '코몽트 기모 터틀넥 5종'], dtype=object)

In [294]:
p_name_split = []
for name in df.p_name:
    p_name_split.append(name.split())

In [295]:
def jaccard(x,y):
    sent_1 = set(p_name_split[x])
    sent_2 = set(p_name_split[y])
    hap = sent_1 | sent_2
    gyo = sent_1 & sent_2

    jaccard_score = len(gyo)/len(hap)
    return jaccard_score

In [296]:
p_df= pd.DataFrame({'name':df.p_name,'isgroup':0})

In [297]:
for i in range(p_df.shape[0]-1):
    p_df.loc[(i,'isgroup')] = jaccard(i, i+1)

In [298]:
df.p_name

0         테이트 남성 셀린니트3종
1         테이트 여성 셀린니트3종
2         테이트 남성 셀린니트3종
3         테이트 여성 셀린니트3종
4         테이트 남성 셀린니트3종
              ...      
37367     일시불쿠첸압력밥솥 6인용
37368    무이자쿠첸압력밥솥 10인용
37369    일시불쿠첸압력밥솥 10인용
37370     무이자쿠첸압력밥솥 6인용
37371     일시불쿠첸압력밥솥 6인용
Name: p_name, Length: 37372, dtype: object

In [299]:
p_df.isgroup.unique()

array([0.5       , 0.        , 1.        , 0.66666667, 0.6       ,
       0.14285714, 0.375     , 0.42857143, 0.71428571, 0.11111111,
       0.2       , 0.125     , 0.33333333, 0.75      , 0.4       ,
       0.16666667, 0.25      , 0.09090909, 0.1       , 0.55555556,
       0.8       , 0.28571429, 0.3       , 0.57142857, 0.22222222,
       0.07692308, 0.36363636, 0.44444444, 0.625     , 0.08333333,
       0.06666667, 0.07142857, 0.27272727, 0.875     , 0.77777778,
       0.18181818, 0.83333333])

In [300]:
# pd.set_option('display.max_rows', p_df.shape[0]+1)
p_df[(p_df['isgroup']<0.2)&(p_df['isgroup']>0)]

,name,isgroup
52,마리노블 밍크 롱코트,0.142857
128,푸마 코튼 라이크라 드로즈 12종,0.111111
129,푸마 INNO 심리스 롱드로즈 7종,0.111111
130,푸마 드라이셀라인 뉴 런닝 9종,0.111111
131,푸마 코튼 라이크라 드로즈 12종,0.111111
...,...,...
37307,무이자 쿠쿠전기밥솥 10인용(CRP-QS107FG/FS),0.166667
37308,일시불 쿠쿠전기밥솥 10인용 (QS),0.166667
37311,무이자 쿠쿠전기밥솥 10인용(CRP-QS107FG/FS),0.166667
37312,일시불 쿠쿠전기밥솥 10인용 (QS),0.166667


In [307]:
# df로 시도1

idx_dict = {'start':0,'end':0}
dfs = []

for i in range(df.shape[0]-1):
    if i % 100 == 0:
        print(i)
    current_hour = df.loc[i,'hour']
    current_simil = df.loc[i,'simil']
    next_hour = df.loc[i+1,'hour']
    next_simil = df.loc[i+1,'simil']
    exp_list = []
    
    if (current_hour == next_hour) and ((current_simil == next_simil) or (next_simil < 0.1)):
        if np.isnan(df.loc[i,'exp_min']) == True:
            pass
        else: 
            exp_list.append(df.loc[i,'exp_min'])
    else:
        idx_start = i
        idx_end = i + 1
        
        idx_dict['end'] = idx_end
        #####################################################################################################################        
        temp = df[idx_dict['start']:idx_dict['end']][['unit_price','p_group']].groupby('p_group').mean()
        
        temp['total_price'] = df[idx_dict['start']:idx_dict['end']][['total_price','p_group']].groupby('p_group').sum()
        temp['p_name'] = df.loc[idx_start,'p_name']
        temp['broadcast'] = df.loc[idx_start,'broadcast'].strftime("%Y-%m-%d %H")
        
        temp['exp_min'] = sum(exp_list)
        
        idx_dict['start'] = idx_dict['end']
        
        dfs.append(temp)
# temp['p_name'] == ~ 식으로 저장해서 마지막에 모두 다 일괄로 씌워버리는 문제 발생 -> dfs로 처리
# exp_min 해결 X 

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [ ]:
df.merge(dfs[0],dfs[1],how='out')

In [313]:
for i in range(len(dfs)-1):
    if i % 100 == 0:
        print(i,'/',len(dfs)-1)
    temp_df = pd.merge(dfs[i],dfs[i+1],how='outer')
    result_df = pd.merge(temp_df,dfs[i+1],how='outer')
    
    # 문제 - 결과적으로 3개의 merged df가 리턴될 것...

0 / 18957
100 / 18957
200 / 18957
300 / 18957
400 / 18957
500 / 18957
600 / 18957
700 / 18957
800 / 18957
900 / 18957
1000 / 18957
1100 / 18957
1200 / 18957
1300 / 18957
1400 / 18957


C:\Users\user\anaconda3\envs\mulcam\lib\site-packages\pandas\core\reshape\merge.py:1108: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  UserWarning,
C:\Users\user\anaconda3\envs\mulcam\lib\site-packages\pandas\core\reshape\merge.py:1097: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  UserWarning,


1500 / 18957
1600 / 18957
1700 / 18957
1800 / 18957


KeyboardInterrupt: 

In [ ]:
# merge 방법 다시 생각해보기
def my_merge(*args):
    

In [317]:
def func(*args):
    for arg in args:
        print(arg)

In [318]:
func(1,2,3,5)

1
2
3
5
